# <span style="color:gray">ipyrad-analysis toolkit:</span> RAxML

RAxML is one of the most popular tools for inferring phylogenetic trees using maximum likelihood. It is fast even for very large data sets. The documentation for raxml is huge, and there are many options. However, I tend to use the same small number of options very frequently, which motivated me to write the `ipa.raxml()` tool to automate the process of generating RAxml command line strings, running them, and accessing the resulting tree files. The simplicity of this tool makes it easy to incorporate into other more complex tools, for example, to infer tress in sliding windows along the genome using the `ipa.treeslider` tool.



### Required software

In [3]:
# conda install ipyrad -c conda-forge -c bioconda
# conda install raxml -c conda-forge -c bioconda
# conda install ipcoal -c conda-forge

In [4]:
import ipyrad.analysis as ipa
import toytree
import ipcoal

### Simulate example dataset

In [19]:
tree = toytree.rtree.imbtree(ntips=10, treeheight=1e7)
tree.draw(ts='p');
mod = ipcoal.Model(tree=tree, Ne=1e5, nsamples=2, mut=1e-8, substitution_model={"kappa": 0.5})
mod.sim_loci(nloci=1, nsites=10000)
mod.write_concat_to_phylip(name="raxtest", outdir="/tmp", diploid=True)

wrote concat locus (10 x 10000bp) to /tmp/raxtest.phy


<svg class="toyplot-canvas-Canvas" height="260.0px" id="te4a253d3539b42ad9c30bbe82dbfab0a" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 350.0 260.0" width="350.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 r0 r1 r2 r3 r4 r5 r6 r7 r8 r9 0 5000000 10000000

### Infer a ML tree

In [20]:
# init raxml object with input data and (optional) parameter options
rax = ipa.raxml(data="/tmp/raxtest.phy", T=4, N=10)

# print the raxml command string for prosperity
print(rax.command)

# run the command, (options: block until finishes; overwrite existing)
rax.run(block=True, force=True)

/home/deren/miniconda3/envs/py38/bin/raxmlHPC-PTHREADS-AVX2 -f a -T 4 -m GTRGAMMA -n test -w /home/deren/Documents/ipyrad/testdocs/analysis/analysis-raxml -s /tmp/raxtest.phy -p 54321 -N 10 -x 12345
job test finished successfully


### Draw the inferred tree
After inferring a tree you can then visualize it in a notebook using `toytree`. 

In [37]:
# load from the .trees attribute of the raxml object, or from the saved tree file
tre = toytree.tree(rax.trees.bipartitions)

# draw the tree
rtre = tre.root("r9", resolve_root_dist=False)
rtre.draw(tip_labels_align=True, node_sizes=18, node_labels="support");

<svg class="toyplot-canvas-Canvas" height="275.0px" id="t5b7525107ef744db98acf0bcb2f71ddf" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 260.0 275.0" width="260.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> 100 100 100 100 100 100 100 100 r1 r0 r2 r3 r4 r5 r6 r7 r8 r9

### Setting parameters

By default several parameters are pre-set in the raxml object. To remove those parameters from the command string you can set them to None. Additionally, you can build complex raxml command line strings by adding almost any parameter to the raxml object init, like below. You probably can't do everythin in raxml using this tool, it's only meant as a convenience. You can always of course just write the raxml command line string by hand instead.

In [38]:
# init raxml object
rax = ipa.raxml(data="/tmp/raxtest.phy", T=4, N=10)

# parameter dictionary for a raxml object
rax.params

N        10                  
T        4                   
binary   ~/miniconda3/envs/py38/bin/raxmlHPC-PTHREADS-AVX2
f        a                   
m        GTRGAMMA            
n        test                
p        54321               
s        /tmp/raxtest.phy    
w        ~/Documents/ipyrad/testdocs/analysis/analysis-raxml
x        12345               

In [39]:
# paths to output files produced by raxml inference
rax.trees

bestTree                   ~/Documents/ipyrad/testdocs/analysis/analysis-raxml/RAxML_bestTree.test
bipartitions               ~/Documents/ipyrad/testdocs/analysis/analysis-raxml/RAxML_bipartitions.test
bipartitionsBranchLabels   ~/Documents/ipyrad/testdocs/analysis/analysis-raxml/RAxML_bipartitionsBranchLabels.test
bootstrap                  ~/Documents/ipyrad/testdocs/analysis/analysis-raxml/RAxML_bootstrap.test
info                       ~/Documents/ipyrad/testdocs/analysis/analysis-raxml/RAxML_info.test

### Cookbook

Most frequently used: perform 100 rapid bootstrap analyses followed by 10 rapid hill-climbing ML searches from random starting trees under the GTRGAMMA substitution model. 

In [41]:
rax = ipa.raxml(
    data="/tmp/raxtest.phy",
    name="test",
    workdir="analysis-raxml",
    m="GTRGAMMA",
    T=8,
    f="a",
    N=50,
)
print(rax.command)
rax.run(force=True)

/home/deren/miniconda3/envs/py38/bin/raxmlHPC-PTHREADS-AVX2 -f a -T 8 -m GTRGAMMA -n test -w /home/deren/Documents/ipyrad/testdocs/analysis/analysis-raxml -s /tmp/raxtest.phy -p 54321 -N 50 -x 12345
job test finished successfully


Another common option: Perform N rapid hill-climbing ML analyses from random starting trees, with no bootstrap replicates. Be sure to use the `BestTree` output from this analysis since it does not produce a `bipartitions` output file. 

In [42]:
rax = ipa.raxml(
    data="/tmp/raxtest.phy",
    name="test",
    workdir="analysis-raxml",
    m="GTRGAMMA",
    T=8,
    f="d",
    N=10,
    x=None,
)
print(rax.command)
rax.run(force=True)

/home/deren/miniconda3/envs/py38/bin/raxmlHPC-PTHREADS-AVX2 -f d -T 8 -m GTRGAMMA -n test -w /home/deren/Documents/ipyrad/testdocs/analysis/analysis-raxml -s /tmp/raxtest.phy -p 54321 -N 10
job test finished successfully
